In [ ]:
import numpy as np
t=[]
for i in range(0,17872):
    try:
        w=np.load(f"content/Kiti-Single/{i}.npy")
    except:
        t.append(i)
print(t) #if output is empty then everything is fine

[]


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv3D, Conv3DTranspose, BatchNormalization, LeakyReLU, Dropout, ReLU, Input, Concatenate, ZeroPadding3D
from tensorflow.keras import Model, Sequential

output_channel = 3

def downsample(filters,size,shape,apply_batchnorm = True):

    initializer = tf.random_normal_initializer(0., 0.02)

    result = Sequential()

    result.add(Conv3D(filters, size, strides=(1,2,2), padding= 'same', batch_input_shape= shape, kernel_initializer= initializer, use_bias= False))

    if apply_batchnorm:
        result.add(BatchNormalization())

    result.add(LeakyReLU())

    return result

def upsample(filters, size, shape, apply_dropout = False):

    initializer = tf.random_normal_initializer(0.,0.02)

    result = Sequential()

    result.add(Conv3DTranspose(filters,size,strides=(1,2,2),padding='same',batch_input_shape=shape,kernel_initializer=initializer, use_bias=False))

    result.add(BatchNormalization())

    if apply_dropout:

        result.add(Dropout(0.5))

    result.add(ReLU())

    return result

def build_generator():

    inputs = Input(shape=[7,256,256,3])

    down_stack = [
        downsample(64,(1,4,4),(None,7,256,256,3),apply_batchnorm=False),
        downsample(128,(1,4,4), (None,7, 128,128,64)),
        downsample(256, (1,4,4), (None,7,64,64,128)),
        downsample(512,(1,4,4),(None,7,32,32,256)),
        downsample(512,(1,4,4),(None,7,16,16,512)),
        downsample(512,(1,4,4),(None,7,8,8,512)),
        downsample(512,(1,4,4),(None,7,4,4,512)),
        downsample(512,(1,4,4),(None,7,2,2,512))
    ]

    up_stack = [
        upsample(512,(1,4,4),(None,7,1,1,512),apply_dropout=True),
        upsample(512,(1,4,4),(None,7,2,2,1024),apply_dropout=True),
        upsample(512,(1,4,4),(None,7,4,4,1024),apply_dropout=True),
        upsample(512,(1,4,4),(None,7,8,8,1024)),
        upsample(256,(1,4,4),(None,7,16,16,1024)),
        upsample(128,(1,4,4),(None,7,32,32,512)),
        upsample(64,(1,4,4),(None,7,64,64,256))
    ]

    initializer = tf.random_normal_initializer(0.,0.02)

    last = Conv3DTranspose(output_channel,(1,4,4),strides=(1,2,2),padding='same',kernel_initializer=initializer,activation='tanh')

    x = inputs

    skips = []

    for down in down_stack:

        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    for up, skip, in zip(up_stack,skips):

        x= up(x)
        x = Concatenate()([x, skip])

    x = last(x)

    return Model(inputs = inputs, outputs = x)

generator = build_generator()
generator.summary()

def downs(filters,size,apply_batch_norm = True):

    initializer = tf.random_normal_initializer(0.,0.02)

    result = Sequential()

    result.add(Conv3D(filters, size, strides=2,padding='same',kernel_initializer=initializer, use_bias=False))

    if apply_batch_norm:

        result.add(BatchNormalization())

    result.add(LeakyReLU())

    return result

def build_discriminator():

    initializer = tf.random_normal_initializer(0.,0.02)

    inp = Input(shape=[7,256,256,3],name='input_img')
    tar = Input(shape=[7,256,256,3],name='target_img')

    x = Concatenate()([inp, tar])

    down1 = downs(64,(1,4,4),False)(x)
    down2 = downs(128,(1,4,4))(down1)
    down3 = downs(256,(1,4,4))(down2)

    zero_pad1 = ZeroPadding3D()(down3)

    conv = Conv3D(512,(1,4,4),strides=1,kernel_initializer=initializer,use_bias=False)(zero_pad1)

    batchnorm1 = BatchNormalization()(conv)

    leaky_relu = LeakyReLU()(batchnorm1)

    zero_pad2 = ZeroPadding3D()(leaky_relu)

    last = Conv3D(1,(1,4,4),strides=1,kernel_initializer=initializer)(zero_pad2)

    return Model(inputs = [inp,tar], outputs = last)

discriminator = build_discriminator()

discriminator.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7, 256, 256, 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 7, 128, 128,  3072        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 7, 64, 64, 12 131584      sequential[0][0]                 
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 7, 32, 32, 25 525312      sequential_1[0][0]               
______________________________________________________________________________________________

In [ ]:
import tensorflow as tf
import os
import time
import datetime
import matplotlib.pyplot as plt
import random

loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

path = 'content/Kiti-Single'

LAMBDA = 100
epochs = 100

def gen_loss(disc_generated_output,gen_output,target):

    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    loss = tf.abs(target - gen_output)
    l1_loss = tf.reduce_mean(loss)
    total_gen_loss = gan_loss + (LAMBDA*l1_loss)

    return total_gen_loss,gan_loss,l1_loss

def  discriminator_loss(disc_real_output,disc_generated_output):

    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

    toal_disc_loss = real_loss + generated_loss

    return toal_disc_loss

generator_optimizer = tf.keras.optimizers.Adam(2e-6,beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-6, beta_1=0.5)

checkpoint_dirs = '/content/drive/MyDrive/Pix2PixforFuture'
checkpoint_prefix = os.path.join(checkpoint_dirs,"ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer, discriminator_optimizer = discriminator_optimizer, generator = generator, discriminator = discriminator)

def generated_img(model,test_input,tar):

    prediction = model(test_input,training = True)
    plt.figure(figsize=(15,15))

    display_list = [test_input[0], tar[0], prediction[0]]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']

    for i in range(3):

        plt.subplot(1,3,i+1)
        plt.title(title[i])
        plt.imshow(display_list[i]*0.5 + 0.5)
        plt.axis('off')

    plt.show()

log_dir = "/content/drive/MyDrive/Pix2PixforFuture/logs"

summary_writer = tf.summary.create_file_writer(log_dir + 'fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function

def train_step(input_img, target, epoch):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

        gen_output = generator(input_img, training = True)

        disc_real_output = discriminator([input_img,target], training = True)
        disc_generated_output = discriminator([input_img,gen_output], training = True)

        gen_total_loss, gen_gan_loss, gen_l1_loss = gen_loss(disc_generated_output,gen_output,target)

        disc_loss = discriminator_loss(disc_real_output,disc_generated_output)

    generator_gradients = gen_tape.gradient(gen_total_loss,generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss,discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients,generator.trainable_variables))
    discriminator_optimizer.apply_gradients((zip(discriminator_gradients,discriminator.trainable_variables)))

    with summary_writer.as_default():

        tf.summary.scalar('gen_total_loss',gen_total_loss, epoch)
        tf.summary.scalar('gen_gan_loss', gen_gan_loss,step=epoch)
        tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
        tf.summary.scalar('disc_loss', disc_loss, step=epoch)

def fit(epochs):

    batch_size = 4
    for epoch in range(epochs):

        start = time.time()

        print("Epoch {}".format(epoch))
        for _,_,files in os.walk(path):
          random.shuffle(files)
          total_input = np.zeros((batch_size,7,256,256,3))
          total_output = np.zeros((batch_size,7,256,256,3))
          num = 0
      
          for k,file in enumerate(files):
              m = int(file.split('.')[0])
              # print(m)
              if m <17850: 
                # print("Loopy")
                
                xm = np.load(path + "/{}.npy".format(m))
                xm_plus1 = np.load(path + "/{}.npy".format(m+1))
                xm_plus2 = np.load(path + "/{}.npy".format(m+2))
                xm_plus3 = np.load(path + "/{}.npy".format(m+3))
                xm_plus4 = np.load(path + "/{}.npy".format(m+4))
                xm_plus5 = np.load(path + "/{}.npy".format(m+5))
                xm_plus6 = np.load(path + "/{}.npy".format(m+6))
                xm_plus7 = np.load(path + "/{}.npy".format(m+7))
                xm_plus8 = np.load(path + "/{}.npy".format(m+8))
                xm_plus9 = np.load(path + "/{}.npy".format(m+9))
                xm_plus10 = np.load(path + "/{}.npy".format(m+10))
                xm_plus11 = np.load(path + "/{}.npy".format(m+11))
                xm_plus12 = np.load(path + "/{}.npy".format(m+12))
                xm_plus13 = np.load(path + "/{}.npy".format(m+13))

                input = np.reshape(np.concatenate([xm,xm_plus1,xm_plus2,xm_plus3,xm_plus4,xm_plus5,xm_plus6]),(7,256,256,3)).astype(np.float32)
                output = np.reshape(np.concatenate([xm_plus7,xm_plus8,xm_plus9,xm_plus10,xm_plus11,xm_plus12,xm_plus13]),(7,256,256,3)).astype(np.float32)

                total_input[num] = input/255.0
                total_output[num] = output/255.0

                num = num + 1

                if k%1000==0: 
                    print(k)
                if num == batch_size:
                  num = 0
                  total_input = tf.convert_to_tensor(total_input,dtype=tf.float32)
                  total_output = tf.convert_to_tensor(total_output,dtype=tf.float32)
                  train_step(total_input, total_output, epoch)
                  total_input = np.zeros((batch_size,7,256,256,3))
                  total_output = np.zeros((batch_size,7,256,256,3))
        print("Time taken for epoch {} is {} sec \n".format(epoch+1, time.time()-start))

        checkpoint.save(file_prefix=checkpoint_prefix)

print("Started Restoring")
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dirs))
print("Done Restoring")
fit(epochs)

Started Restoring
Done Restoring
Epoch 0
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
Time taken for epoch 1 is 3362.5772783756256 sec 

Epoch 1
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
Time taken for epoch 2 is 2915.188324689865 sec 

Epoch 2
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
Time taken for epoch 3 is 2913.2279963493347 sec 

Epoch 3
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
Time taken for epoch 4 is 2917.6775209903717 sec 

Epoch 4
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
Time taken for epoch 5 is 2914.7374143600464 sec 

Epoch 5
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
Time taken for epoch 6 is 2912.5883407592773 sec 

Epoch 6
0
1000
2000
3000
4000
5000
6000
7000

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dirs))

In [ ]:
m = 12230
xm = np.load(path + "/{}.npy".format(m))
xm_plus1 = np.load(path + "/{}.npy".format(m+1))
xm_plus2 = np.load(path + "/{}.npy".format(m+2))
xm_plus3 = np.load(path + "/{}.npy".format(m+3))
xm_plus4 = np.load(path + "/{}.npy".format(m+4))
xm_plus5 = np.load(path + "/{}.npy".format(m+5))
xm_plus6 = np.load(path + "/{}.npy".format(m+6))

input = np.reshape(np.concatenate([xm,xm_plus1,xm_plus2,xm_plus3,xm_plus4,xm_plus5,xm_plus6]),(7,256,256,3)).astype(np.float32)

total_input = tf.expand_dims(input, axis = 0)

gen_output = generator(total_input/255.0, training = False)

In [ ]:
print(gen_output[0][1].shape)

(256, 256, 3)


In [ ]:
from google.colab.patches import cv2_imshow
#gen_output = tf.squeeze(gen_output, axis=0)

for i in range(7):
  cv2_imshow(input[i])

print("GO")

for i in range(7):
  cv2_imshow(gen_output[0][i].numpy()*255)